## Demo Azure opan Ai assistant(s) - basic math example

We have to import the python libraries we need in order to launch the Azure open Ai Assistant

We will use the openai libray for access, os, json, dotenv for loading secrets key in secrets.env :
    
    pip install openai
    pip install python-dotenv


In [ ]:
!pip install --upgrade pip
%pip install python-dotenv
%pip install openai --upgrade
%pip install time


We have to import the python libraries we need in order to launch the Azure open Ai Assistant

In [13]:
import os
import time
import json
from dotenv import load_dotenv
from openai import AzureOpenAI

In a first place you have to place in secrets.env you api key for Azure open AI, your endpoint and the name of deployment for the model you want to use.

We are loading the secrets for the access to the model with the param found in the secrets.env file

In [26]:
load_dotenv('secrets.env')
azure_openaikey = os.getenv("AZURE_OPENAIKEY")
azure_openaiendpoint = os.getenv("AZURE_OPENAIENDPOINT")
azure_openaideployment = os.getenv("AZURE_OPENAIDEPLOYMENT")

We could now instantiate the AzureOpenAi client

In [28]:
client = AzureOpenAI(
    api_key= azure_openaikey,
    api_version="2024-02-15-preview", ## this is a preview version of the api ( not documented yet)
    azure_endpoint = azure_openaiendpoint
    )

The apis swagger is available at the following url [https://github.com/Azure/azure-rest-api-specs/tree/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/preview](https://github.com/Azure/azure-rest-api-specs/tree/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/preview)

Launch the assistant based on the client with the model desired

In [29]:

# Create an assistant
assistant = client.beta.assistants.create(
    name="Math Assist",
    instructions="You are an AI assistant that can write code to help answer math questions.",
    tools=[{"type": "code_interpreter"}],
    model=azure_openaideployment #You must replace this value with the deployment name for your model.
)


In [30]:

# Create a thread
thread = client.beta.threads.create()

In [31]:

# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user", ## actually the only role available - stay tuned !
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [32]:

# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [33]:

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status

In [34]:
# Wait till the assistant has responded
while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    status = run.status

messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [35]:

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_5Oc2nmN7boHKfYuJaOnxkYrQ",
      "assistant_id": "asst_vid43QSBUFAVr7f7nraAwfkS",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "The solution to the equation \\(3x + 11 = 14\\) is \\(x = 1\\)."
          },
          "type": "text"
        }
      ],
      "created_at": 1706827368,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_VlCTYceNmV8HlQyOLQUe7Xj6",
      "thread_id": "thread_KM402gTevvYxlfxYA1ONQPHP"
    },
    {
      "id": "msg_ZHWVk1gJT292L4YJzlbLjmFq",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "I need to solve the equation `3x + 11 = 14`. Can you help me?"
          },
          "type": "text"
        }
      ],
      "created_at": 1706827327,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
  